<a href="https://colab.research.google.com/github/Abhilaksh1498/Everything-ML-done_by_me-/blob/master/Testing_NER_model_%2B_Mounting_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import tensorflow
import numpy as np
from tensorflow.keras.models import load_model
import pickle, json

In [6]:
import json
import keras_contrib as kcb

In [7]:
embeddings_matrix = pickle.load(open('/content/gdrive/MyDrive/weights/embeddings_matrix.pkl', 'rb'))
idx2tag = pickle.load(open('/content/gdrive/MyDrive/weights/idx2tag.pkl', 'rb'))
with open('/content/gdrive/MyDrive/weights/cust_vocab.json') as f:
  word2idx = json.load(f)

In [8]:
print(f'Total words in vocabulary: {len(word2idx.keys())}')
print(f'Total tags: {len(idx2tag.keys())}')

Total words in vocabulary: 11096
Total tags: 18


In [12]:
import keras as k

In [13]:
model = k.models.load_model('/content/gdrive/MyDrive/weights/cust_emb_model.h5', 
                            custom_objects = {'CRF': kcb.layers.CRF,
                                              'crf_loss': kcb.losses.crf_loss,
                                              'crf_accuracy': kcb.metrics.crf_accuracy})

In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 23)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 23, 100)           1109700   
_________________________________________________________________
bidirectional_lstm1 (Bidirec (None, 23, 200)           160800    
_________________________________________________________________
bidirectional_lstm2 (Bidirec (None, 23, 400)           641600    
_________________________________________________________________
timedistributed (TimeDistrib (None, 23, 18)            7218      
_________________________________________________________________
crf_1 (CRF)                  (None, 23, 18)            702       
Total params: 1,920,020
Trainable params: 1,920,020
Non-trainable params: 0
_________________________________________________

In [14]:
model_glove = k.models.load_model('/content/gdrive/MyDrive/weights/glove_model.h5', 
                            custom_objects = {'CRF': kcb.layers.CRF,
                                              'crf_loss': kcb.losses.crf_loss,
                                              'crf_accuracy': kcb.metrics.crf_accuracy})

In [15]:
model_glove.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 25)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 25, 200)           80000600  
_________________________________________________________________
bidirectional_lstm1 (Bidirec (None, 25, 400)           641600    
_________________________________________________________________
bidirectional_lstm2 (Bidirec (None, 25, 800)           2563200   
_________________________________________________________________
timedistributed (TimeDistrib (None, 25, 18)            14418     
_________________________________________________________________
crf_2 (CRF)                  (None, 25, 18)            702       
Total params: 83,220,520
Trainable params: 3,219,920
Non-trainable params: 80,000,600
_______________________________________

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pred2label(pred, idx2tag):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        
        out.append(out_i)
    
    return out

def preprocess_request(sent, word2idx):
    sent_x = []
    for s in [sent]:
        temp = []
        for w in s.split():
            if word2idx.get(w.lower()) != None:
                temp.append(word2idx[w.lower()])
            else:
                temp.append(word2idx['<unk>'])
        
        sent_x.append(temp)

    pad_x = pad_sequences(maxlen=25, sequences=sent_x, padding="post", value=word2idx["<unk>"])

    return sent_x, pad_x


In [24]:
query = input('Enter a Sentence')
sent_x , padded_sent = preprocess_request(query, word2idx)
pred = model_glove.predict(padded_sent)
p = pred2label(pred, idx2tag)

p = np.array(p)

Enter a Sentence'Mark Zuckerberg is one of the founders of Facebook, a company from the United States'
[['I-gpe' 'I-eve' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-tim' 'I-gpe' 'B-geo'
  'O' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe'
  'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe' 'I-gpe']]


In [26]:
print(p[0, : len(sent_x[0])].tolist())

['I-gpe', 'I-eve', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'I-tim', 'I-gpe', 'B-geo', 'O', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe']


In [59]:
def predict():
  query = input('Enter a Sentence: ')
  sent_x , padded_sent = preprocess_request(query, word2idx)
  pred = model_glove.predict(padded_sent)
  p = pred2label(pred, idx2tag)
  p = np.array(p)
  return p[0, : len(sent_x[0])].tolist()

In [33]:
print(predict())

Enter a Sentence'Air  inlet  inspected  metal  rust  observed'
['I-eve', 'I-gpe', 'I-gpe', 'B-geo', 'I-tim', 'I-gpe']


In [34]:
# try a sentence it was trained on
print(predict())           # glove_model o/p

Enter a Sentence'Carry out borescope inspection of turbine in accordance with Siemens procedure referenced in the relevant overhaul manual'
['I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'B-geo', 'O', 'I-gpe', 'B-geo', 'I-gpe', 'I-gpe', 'B-geo', 'I-gpe', 'I-gpe', 'B-geo', 'O', 'I-gpe']


In [58]:
# Trying the custom_emb model
def predict_custom():
  query = input('Enter a Sentence: ')
  sent_x , padded_sent = preprocess_request(query, word2idx)
  pred = model.predict(padded_sent)
  p = pred2label(pred, idx2tag)
  p = np.array(p)
  return p[0, : len(sent_x[0])].tolist()


In [41]:
print(predict_custom())   # custom_emb o/p

Enter a Sentence'Mark Zuckerberg is one of the founders of Facebook, a company from the United States'
['B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-art', 'B-eve', 'B-art', 'B-eve', 'B-art']


In [45]:
print(predict())          # glove_model o/p

Enter a Sentence'Mark Zuckerberg is one of the founders of Facebook, a company from the United States'
['I-gpe', 'I-eve', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'I-tim', 'I-gpe', 'B-geo', 'O', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe', 'I-gpe']


In [63]:
# modifying the already written predict function to return tuples of NE and classes
def predict_custom():             # for sutom_emb model
  query = input('Enter a Sentence: ')
  sent_x , padded_sent = preprocess_request(query, word2idx)
  pred = model.predict(padded_sent)
  p = pred2label(pred, idx2tag)
  p = np.array(p)
  temp = zip(query.split()[:len(sent_x[0])], p[0, : len(sent_x[0])].tolist()) 
  result = []
  for x in temp:
    if x[1] != 'O':
      result.append(x)
  return result


def predict():                   # for glove model
  query = input('Enter a Sentence: ')
  sent_x , padded_sent = preprocess_request(query, word2idx)
  pred = model_glove.predict(padded_sent)
  p = pred2label(pred, idx2tag)
  p = np.array(p)
  temp = zip(query.split()[:len(sent_x[0])], p[0, : len(sent_x[0])].tolist()) 
  result = []
  for x in temp:
    if x[1] != 'O':
      result.append(x)
  return result

In [64]:
print(predict())          # glove_model o/p

Enter a Sentence: 'Mark Zuckerberg is one of the founders of Facebook, a company from the United States'
[("'Mark", 'I-gpe'), ('Zuckerberg', 'I-eve'), ('is', 'I-gpe'), ('one', 'I-gpe'), ('of', 'I-gpe'), ('the', 'I-gpe'), ('founders', 'I-tim'), ('of', 'I-gpe'), ('Facebook,', 'B-geo'), ('company', 'I-gpe'), ('from', 'I-gpe'), ('the', 'I-gpe'), ('United', 'I-gpe'), ("States'", 'I-gpe')]


In [65]:
print(predict())

Enter a Sentence: 'Kate  lives  on  Mars'
[("'Kate", 'I-gpe'), ('lives', 'B-geo'), ('on', 'I-gpe'), ("Mars'", 'I-gpe')]
